In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

llm

c:\Users\mydhi\OneDrive\Desktop\LangchainProjects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x0000029698347770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000029696276630>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
os.environ['HF_TOKEN']=os.getenv("HUGGINGFACE_API")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

TypeError: str expected, not NoneType

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
import bs4
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs=loader.load()
docs

In [ ]:
txt_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks=txt_splitter.split_documents(docs)
db=Chroma.from_documents(chunks,embeddings)
retriever=db.as_retriever()
retriever





In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
prompt=ChatPromptTemplate.from_messages([("sysytem",system_prompt),("human","{input}")])

question_answer_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
rag_retrieval_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"What is Self-Reflection"})
response['answer']

In [ ]:
rag_chain.invoke({"input":"Howw do we achieve it"})

## NOW ADDING CHATHISTORY

from langchain.chains import create_history_aware_retriever
from lanhchain_core.prompts import MessagesPalceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt=ChatPromptTemplate.from_messages([("sysytem":contextualize_q_system_prompt),MessagesPlaceholder("chat_history"),("human":"{input}")])
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

In [ ]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
qa_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,qa_chain)


In [ ]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Self-Reflection"
res1=rag_chain.invoke({"input":question,"chat_history":chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=res1["answer"])
    ]
)

question2="Tell me more about it?"
res2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(res2['answer'])
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=res2["answer"])
    ]
)

In [ ]:
chat_history

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain=RunnableWithMessageHistory(rag_chain,get_session_history,
                                                    input_messages_key="input",
                                                    output_messages_key="answer",history_messages_key='chat_history')

In [ ]:
conversational_rag_chain.invoke({"input":"what si the task decomposition"},
                                config={"configurable":{"session_id": "abc123"}})["answer"]

In [ ]:
store